In [26]:
from datasets import load_dataset
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = load_dataset('0jl/SPair-71k', name='data', split='train')

In [ ]:
distances_from_corner = []
for x in tqdm(data):
    w, h = x['image_width'], x['image_height']
    for kp in x['kps']:
        if kp[0] < 0 or kp[1] < 0: continue
        distances_from_corner.append(min(max(abs(kp[0]-i), abs(kp[1]-j)) for i, j in [(0,0), (0,h-1), (w-1,0), (w-1, h-1)]))
distances_from_corner = np.array(distances_from_corner)

In [ ]:
plt.hist(distances_from_corner, bins=100)
plt.show()

In [ ]:
sizes = [8, 16, 32, 64]
changes = []
for size in sizes:
    corner_kps = (distances_from_corner < size).sum()
    avg_kp_per_token = len(distances_from_corner) / (512/size)**2
    print(f'size: {size}')
    print(f'corner tokens: {corner_kps} ({corner_kps / len(distances_from_corner):.5%})')
    print(f'avg kps per token: {avg_kp_per_token:.5} ({1 / (512/size)**2:.5%})')
    print(f'change in kps for corners: {(corner_kps/4) / avg_kp_per_token:.5} - {avg_kp_per_token / (corner_kps/4):.5}')
    changes.append((corner_kps/4) / avg_kp_per_token)

plt.plot(sizes, changes)
plt.xscale('log')
plt.xlabel('token size')
plt.ylabel('change in kps for corners')
plt.xticks(sizes, [f'{s}x{s}' for s in sizes])
plt.minorticks_off()
plt.show()
